In [ ]:
import os
import numpy as np
import glob as glob
import pandas as pd
from pathlib import Path
from scipy.io import loadmat
import matplotlib.pyplot as plt
import math
import seaborn as sns
import scipy 
from matplotlib.ticker import MultipleLocator
import sklearn
from sklearn.neighbors import KernelDensity
from sklearn.model_selection import GridSearchCV
from scipy.ndimage import gaussian_filter1d

# Import Data

In [ ]:
All_stopdata = pd.read_parquet(r'C:\1_Stop_project_allData\Ball_fitting_V2\data\df_preproc.parquet', engine='pyarrow') ## path to parquet file with rawe data for all Stops

In [ ]:
## Define genotypes
BRK = All_stopdata.loc[All_stopdata['genotype'].str.contains('Stop')]
FG = All_stopdata.loc[All_stopdata['genotype'].str.contains('FG')]
BB = All_stopdata.loc[All_stopdata['genotype'].str.contains('BB')]
ES = All_stopdata.loc[All_stopdata['genotype'].str.contains('ES')]

In [ ]:
## Remove flies with tracking errors in hind leg from hind leg analyses
BRK = BRK.loc[(BRK['flynum']!= 25)]
HL_prune = [1,3,4,5,7,16,18,19,22,25, 37]
BRK_HL_pruned = BRK[~BRK.flynum.isin(HL_prune)]

In [ ]:
# Extracting only 66Hz stim data
BRK_66 =BRK.groupby('SF').get_group(66).reset_index(drop=True)
FG_66 =FG.groupby('SF').get_group(66).reset_index(drop=True)
BB_66 =BB.groupby('SF').get_group(66).reset_index(drop=True)
ES_66 =ES.groupby('SF').get_group(66).reset_index(drop=True)
BRK_HL_66 = BRK_HL_pruned.groupby('SF').get_group(66).reset_index(drop=True)

# Functions

In [ ]:
L1_TaG_idx = [62,63,64]
R1_TaG_idx = [17,18,19]
L2_TaG_idx = [77,78,79]
R2_TaG_idx = [32,33,34]
L3_TaG_idx = [92,93,94]
R3_TaG_idx = [47,48,49]

In [ ]:
def filter_prestim_vel (dataframe):
    """filter dataframe by pre-stim fwd vel (x_vel) in 80 frames immediately preceeding stim; meanvel > 0.5mm/s

    Parameters
    ----------
    dataframe : _type_
        

    Returns
    -------
    dataframe
        filetered for trials with sufficient pre-stim x_vel
    """
    threshold = 1
    filtered_df = pd.DataFrame()
    for n in dataframe['flynum'].unique().tolist():
        for t in dataframe.groupby('flynum').get_group(n)['tnum'].unique().tolist():
            data = dataframe.groupby('flynum').get_group(n).groupby('tnum').get_group(t)
            mean_vel = np.mean(data.loc[(data['fnum']%1400>320)&(data['fnum']%1400<400)]['x_vel'])
            if mean_vel >=threshold:
                filtered_df = pd.concat([filtered_df.reset_index(drop=True), data.reset_index(drop=True)], axis = 0)
    return filtered_df

In [ ]:
def plot_heatmaps(df, title):
    """
    # Plots Fe-Ti joint angle as a heatmap

    Parameters
    ----------
    df : dataframe
    title : title for plot
    """
    plt.rcParams['figure.figsize'] = (10,7)
    fig, ax = plt.subplots(3,2, sharex = True, sharey = True)
    vmin = 0
    vmax = 150
    cmap = 'coolwarm'
    

    sns.heatmap(ax = ax[0,0], data = np.array(df['L1C_flex']).reshape((int(len(df)/1400)), 1400), vmin = vmin, vmax = vmax, cmap = cmap, cbar=False)
    sns.heatmap(ax = ax[1,0], data = np.array(df['L2C_flex']).reshape((int(len(df)/1400)), 1400), vmin = vmin, vmax = 130, cmap = cmap, cbar=False)
    sns.heatmap(ax = ax[2,0], data = np.array(df['L3C_flex']).reshape((int(len(df)/1400)), 1400), vmin = vmin, vmax = vmax, cmap = cmap, cbar=False)

    sns.heatmap(ax = ax[0,1], data = np.array(df['R1C_flex']).reshape((int(len(df)/1400)), 1400), vmin = vmin, vmax = vmax, cmap = cmap, cbar=False)
    sns.heatmap(ax = ax[1,1], data = np.array(df['R2C_flex']).reshape((int(len(df)/1400)), 1400), vmin = vmin, vmax = 130, cmap = cmap, cbar=False)
    sns.heatmap(ax = ax[2,1], data = np.array(df['R3C_flex']).reshape((int(len(df)/1400)), 1400), vmin = vmin, vmax = vmax, cmap = cmap, cbar=False)
    ax[0,0].axvline(x = 400, color = 'k', alpha = 0.7, linestyle = '--')
    ax[1,0].axvline(x = 400, color = 'k', alpha = 0.7, linestyle = '--')
    ax[2,0].axvline(x = 400, color = 'k', alpha = 0.7, linestyle = '--')
    ax[0,1].axvline(x = 400, color = 'k', alpha = 0.7, linestyle = '--')
    ax[1,1].axvline(x = 400, color = 'k', alpha = 0.7, linestyle = '--')
    ax[2,1].axvline(x = 400, color = 'k', alpha = 0.7, linestyle = '--')

    ax[0,0].axvline(x = 1000, color = 'k', alpha = 0.7, linestyle = '--')
    ax[1,0].axvline(x = 1000, color = 'k', alpha = 0.7, linestyle = '--')
    ax[2,0].axvline(x = 1000, color = 'k', alpha = 0.7, linestyle = '--')
    ax[0,1].axvline(x = 1000, color = 'k', alpha = 0.7, linestyle = '--')
    ax[1,1].axvline(x = 1000, color = 'k', alpha = 0.7, linestyle = '--')
    ax[2,1].axvline(x = 1000, color = 'k', alpha = 0.7, linestyle = '--')
    plt.suptitle(title)
    plt.tight_layout()
    # plt.savefig(r'Z:\STOP PROJECT SHARED FOLDER\Fig2\07222023\1_Heatmaps\ES_66Hz.eps', format = 'eps',dpi = 300)
    # plt.savefig(r'Z:\STOP PROJECT SHARED FOLDER\Fig2\07222023\1_Heatmaps\ES_66Hz.tiff',format = 'tiff' ,dpi = 300)
    plt.show()

In [ ]:
def get_swing_at_lightON (temp_filt_df):
    """extract all trials with leg in mid swing at light ON

    Parameters
    ----------
    temp_filt_df : velocity filtered Dataframe

    Returns
    -------
    DataFrame
        all trials with legs in mid-swing at light ON
    """
    L1_swing = pd.DataFrame()
    L2_swing = pd.DataFrame()
    L3_swing = pd.DataFrame()
    R1_swing = pd.DataFrame()
    R2_swing = pd.DataFrame()
    R3_swing = pd.DataFrame()
    for N in temp_filt_df['flynum'].unique().tolist():
        for t in temp_filt_df.groupby('flynum').get_group(N)['tnum'].unique().tolist():
            data = temp_filt_df.groupby('flynum').get_group(N).groupby('tnum').get_group(t).reset_index(drop=True)
            if data['L-F_stepcycle'][400] == False:
                L1_swing = pd.concat([L1_swing.reset_index(drop=True), data.reset_index(drop=True)], axis =0)

            if data['L-M_stepcycle'][400] == False:
                L2_swing = pd.concat([L2_swing.reset_index(drop=True), data.reset_index(drop=True)], axis =0)

            if data['L-H_stepcycle'][400] == False:
                L3_swing = pd.concat([L3_swing.reset_index(drop=True), data.reset_index(drop=True)], axis =0)

            if data['R-F_stepcycle'][400] == False:
                R1_swing = pd.concat([R1_swing.reset_index(drop=True), data.reset_index(drop=True)], axis =0)

            if data['R-M_stepcycle'][400]== False:
                R2_swing = pd.concat([R2_swing.reset_index(drop=True), data.reset_index(drop=True)], axis =0)

            if data['R-H_stepcycle'][400] == False:
                R3_swing = pd.concat([R3_swing.reset_index(drop=True), data.reset_index(drop=True)], axis =0)
    return L1_swing.reset_index(drop=True) , L2_swing.reset_index(drop=True), L3_swing.reset_index(drop=True), R1_swing.reset_index(drop=True), R2_swing.reset_index(drop=True) ,R3_swing.reset_index(drop=True)

In [ ]:
def get_stance_at_lightON (temp_filt_df):
    """extract all trials with leg in mid stance at light ON

    Parameters
    ----------
    temp_filt_df : velocity filtered DataFrame
        

    Returns
    -------
    DataFrame
        all trials with legs in mid-stance at light ON
    """
    L1_stance = pd.DataFrame()
    L2_stance = pd.DataFrame()
    L3_stance = pd.DataFrame()
    R1_stance = pd.DataFrame()
    R2_stance = pd.DataFrame()
    R3_stance = pd.DataFrame()
    for N in temp_filt_df['flynum'].unique().tolist():
        for t in temp_filt_df.groupby('flynum').get_group(N)['tnum'].unique().tolist():
            data = temp_filt_df.groupby('flynum').get_group(N).groupby('tnum').get_group(t).reset_index(drop=True)
            if data['L-F_stepcycle'][400] == True:
                L1_stance = pd.concat([L1_stance.reset_index(drop=True), data.reset_index(drop=True)], axis =0)

            if data['L-M_stepcycle'][400] == True:
                L2_stance = pd.concat([L2_stance.reset_index(drop=True), data.reset_index(drop=True)], axis =0)

            if data['L-H_stepcycle'][400] == True:
                L3_stance = pd.concat([L3_stance.reset_index(drop=True), data.reset_index(drop=True)], axis =0)

            if data['R-F_stepcycle'][400] == True:
                R1_stance = pd.concat([R1_stance.reset_index(drop=True), data.reset_index(drop=True)], axis =0)

            if data['R-M_stepcycle'][400]== True:
                R2_stance = pd.concat([R2_stance.reset_index(drop=True), data.reset_index(drop=True)], axis =0)

            if data['R-H_stepcycle'][400] == True:
                R3_stance = pd.concat([R3_stance.reset_index(drop=True), data.reset_index(drop=True)], axis =0)
    return L1_stance.reset_index(drop=True) , L2_stance.reset_index(drop=True), L3_stance.reset_index(drop=True), R1_stance.reset_index(drop=True), R2_stance.reset_index(drop=True) ,R3_stance.reset_index(drop=True)

In [ ]:
def plot_all_flexion_ang(df, Leg):
    """plot all 4 FLEXION angles in [280 - 500] frames in all trials in DF, with SC predictions overlaid

    Parameters
    ----------
    df : DataFrame
        
    Leg : 'L/Rx'
        
    """
    if Leg == 'L1':
        cols = df.columns[df.columns.str.contains(pat = 'L1')]
        SC = pd.concat([df['flynum'], df['tnum'], df['L-F_stepcycle'].astype(int)], axis =1)
        title = 'Left front leg'
    elif Leg == "L2":
        cols = df.columns[df.columns.str.contains(pat = 'L2')]
        SC = pd.concat([df['flynum'], df['tnum'], df['L-M_stepcycle'].astype(int)], axis =1)
        title = 'Left mid leg'
    elif Leg =="L3":
        cols = df.columns[df.columns.str.contains(pat = 'L3')]
        SC = pd.concat([df['flynum'], df['tnum'], df['L-H_stepcycle'].astype(int)], axis =1)
        title = 'Left hind leg'
    elif Leg == "R1":
        cols = df.columns[df.columns.str.contains(pat = 'R1')]
        SC = pd.concat([df['flynum'], df['tnum'], df['R-F_stepcycle'].astype(int)], axis =1)
        title = 'Right front leg'
    elif Leg == "R2":
        cols = df.columns[df.columns.str.contains(pat = 'R2')]
        SC = pd.concat([df['flynum'], df['tnum'], df['R-M_stepcycle'].astype(int)], axis =1)
        title = 'Right mid leg'
    elif Leg =="R3":
        cols = df.columns[df.columns.str.contains(pat = 'R3')]
        SC = pd.concat([df['flynum'], df['tnum'], df['R-H_stepcycle'].astype(int)], axis =1)
        title = 'Right hind leg'

    
    plt.rcParams['figure.figsize'] = (12,int(len(df)/1000))
    

    
    fig, ax = plt.subplots(int((len(df)/1400)), 4, sharex= True, sharey = 'col')
     
    plt.suptitle(title)
    i = 0
    
    for n in df['flynum'].unique().tolist():
        for t in df.groupby('flynum').get_group(n)['tnum'].unique().tolist():
            data = df.groupby('flynum').get_group(n).groupby('tnum').get_group(t)
            SC_fly = SC.groupby('flynum').get_group(n).groupby('tnum').get_group(t)


            print((n,t))

            ax[i, 0].plot(data[cols[1]][280:500]. reset_index(drop=True))  ## 'L1A_flexion"
            
            ax[0, 0].title.set_text('Thorax-Coxa Flexion')
            
            SC_temp = SC_fly.iloc[280:500, 2]. reset_index(drop=True)

            for swing in SC_temp.loc[SC_temp == False].index.tolist():
                ax[i,0].axvline (x = swing, color = 'grey', alpha= 0.3)
            val_A = data[cols[1]].reset_index(drop=True)[400]
            ax[i, 0].plot(120, val_A, 'o', color ='crimson')

            ax[i, 1].plot(data[cols[3]][280:500]. reset_index(drop=True), label = (n,t))
            ax[i,1].legend()
            
            
            ax[0, 1].title.set_text('Coxa-Femur Flexion')
            
            

            for swing in SC_temp.loc[SC_temp == False].index.tolist():
                ax[i,1].axvline (x = swing, color = 'grey', alpha= 0.3)
            val_B = data[cols[3]].reset_index(drop=True)[400]
            ax[i, 1].plot(120, val_B, 'o', color ='crimson')

            ax[i, 2].plot(data[cols[5]][280:500]. reset_index(drop=True))
            
            
            
            ax[0, 2].title.set_text('Femur-Tibia Flexion')
            
            for swing in SC_temp.loc[SC_temp == False].index.tolist():
                ax[i,2].axvline (x = swing, color = 'grey', alpha= 0.3)
            val_C = data[cols[5]].reset_index(drop=True)[400]
            ax[i, 2].plot(120, val_C, 'o', color ='crimson')

            ax[i, 3].plot(data[cols[7]][280:500]. reset_index(drop=True))
            
            
            ax[0, 3].title.set_text('Tibia-Tarsus Flexion')
            
            for swing in SC_temp.loc[SC_temp == False].index.tolist():
                ax[i,3].axvline (x = swing, color = 'grey', alpha= 0.3)
            val_D = data[cols[7]].reset_index(drop=True)[400]
            ax[i, 3].plot(120, val_D, 'o', color ='crimson')

            i+=1
    plt.tight_layout()
    plt.xticks([0,50,100,150,200], [0, 0.25, 0.5, 0.75, 1.00])
    # plt.savefig(r'Z:\STOP PROJECT SHARED FOLDER\Fig2\07302023_refinedfigs\9_RawJointAngle_wSteps\MidStance\BB_R3stance.svg', dpi = 300)
    plt.show()

In [ ]:
def get_TD_LO(df, Leg):
    if Leg == 'L1':
        SC = df['L-F_stepcycle'].astype(int)
    elif Leg == "L2":
        SC = df['L-M_stepcycle'].astype(int)
    elif Leg =="L3":
        SC = df['L-H_stepcycle'].astype(int)
    elif Leg == "R1":
        SC = df['R-F_stepcycle'].astype(int)
    elif Leg == "R2":
        SC = df['R-M_stepcycle'].astype(int)
    elif Leg =="R3":
        SC = df['R-H_stepcycle'].astype(int)
        
    Leg_diff =  pd.DataFrame(np.diff(np.array(SC)))
    Leg_TD_frame = pd.DataFrame(Leg_diff.index[Leg_diff[0]==1])
    Leg_LO_frame = pd.DataFrame(Leg_diff.index[Leg_diff[0]==-1])
    
    Leg_SC_init = pd.concat([Leg_TD_frame, Leg_LO_frame], axis = 1)
    Leg_SC_init.columns=['TD', 'LO']
    return Leg_SC_init

In [ ]:
# align arrays to get stance
def align_arr(SC_df):
    SC_df_new = pd.DataFrame()
    TD_1 =int(SC_df.iloc[0,0])
    LO_1 =int(SC_df.iloc[0,1])
    if TD_1 < LO_1 : #if the first event is a touchdown off
        TD_new = SC_df.iloc[1:, 0].reset_index(drop=True)
        SC_df_new = pd.concat([TD_new, SC_df.iloc[:,1]], axis=1).dropna()
        SC_df_new.columns = ['TD', 'LO']
    else:
        SC_df_new = SC_df.dropna()
    return SC_df_new

In [ ]:

def swing_dur_change(df_leg_swing , Leg ):
    
    Swingdur_beforelight = []
    Swingdur_wlight = []
    data_with_Nt = []
    for N in df_leg_swing['flynum'].unique().tolist():
        for t in df_leg_swing.groupby('flynum').get_group(N)['tnum'].unique().tolist():
                data = df_leg_swing.groupby('flynum').get_group(N).groupby('tnum').get_group(t).reset_index(drop=True)
                # get TD, LO frames
                data_SC_frames = get_TD_LO(data, Leg)
                if len(data_SC_frames) >0:
                    #get swings initiated before light ON
                    data_SC_Frames_b4lightON=data_SC_frames.loc[data_SC_frames['LO']<400]
                    if len(data_SC_Frames_b4lightON) >0:
                        #get pairs of TD and LO
                        data_SC_frames_aligned = align_arr(data_SC_Frames_b4lightON)
                        if len(data_SC_frames_aligned) >0:
                            # get swing dur
                            # swing_durarr_fly = abs(data_SC_frames_aligned.diff(axis = 1))
                            swing_durarr_fly = data_SC_frames_aligned.diff(axis = 1) * -1
                            
                            swing_dur_fly = swing_durarr_fly.iloc[:-1,1].tolist()
                            
                            swing_dur_med = np.median(swing_dur_fly)  ## median swing duration pre stim
                            
                            swing_w_light = swing_durarr_fly.iloc[len(data_SC_frames_aligned)-1,1] ## swing dur w light
                    
                            # append values to list
                            Swingdur_beforelight.append(swing_dur_med)
                            Swingdur_wlight.append(swing_w_light)
                            data_with_Nt.append([(N,t), swing_w_light])

    Swing_dur_df = pd.concat([pd.DataFrame(Swingdur_beforelight), pd.DataFrame(Swingdur_wlight)], axis =1)
    Swing_dur_df.columns = ['before_light', 'lightON']
    return Swing_dur_df, data_with_Nt

#############################
def plot_swing_dur (swing_dur_df):
    fig = plt.figure()
    fig.set_figwidth(6)
    fig.set_figheight(5)
    for i in range(len(swing_dur_df)):
        if swing_dur_df.iloc[i,0] < 25:
            plt.plot(['before', 'after'], [swing_dur_df.iloc[i,0], swing_dur_df.iloc[i,1]], '.', color="#000000")    
            plt.plot(['before', 'after'], [swing_dur_df.iloc[i,0], swing_dur_df.iloc[i,1]])
        
    plt.show()  

In [ ]:
def plot_swing_dur_alllegs (swing_dur_F, swing_dur_M, swing_dur_H):
    plt.rcParams['figure.figsize'] = (10,3)
    fig, ax = plt.subplots(1,3)

    for i in range(len(swing_dur_F)):
        if swing_dur_F.iloc[i,0] < 25:
            ax[0].plot(['pre-stim', 'stim'], [swing_dur_F.iloc[i,0], swing_dur_F.iloc[i,1]], '.', color="#000000")    
            ax[0].plot(['pre-stim', 'stim'], [swing_dur_F.iloc[i,0], swing_dur_F.iloc[i,1]], color = 'salmon')
            ax[0].title.set_text('Front legs')
    
    for i in range(len(swing_dur_M)):
        if swing_dur_M.iloc[i,0] < 25:
            ax[1].plot(['pre-stim', 'stim'], [swing_dur_M.iloc[i,0], swing_dur_M.iloc[i,1]], '.', color="#000000")    
            ax[1].plot(['pre-stim', 'stim'], [swing_dur_M.iloc[i,0], swing_dur_M.iloc[i,1]],  color = 'skyblue')
            ax[1].title.set_text('Mid legs')
    
    for i in range(len(swing_dur_H)):
        if swing_dur_H.iloc[i,0] < 25:
            ax[2].plot(['pre-stim', 'stim'], [swing_dur_H.iloc[i,0], swing_dur_H.iloc[i,1]], '.', color="#000000")    
            ax[2].plot(['pre-stim', 'stim'], [swing_dur_H.iloc[i,0], swing_dur_H.iloc[i,1]], color = 'olive')
            ax[2].title.set_text('Hind legs')

    # plt.savefig(r'C:\1_Stop_project_allData\Figures\07092023\1_swingduration\FG_66Hz.png', dpi = 300)
    # plt.savefig(r'Z:\STOP PROJECT SHARED FOLDER\Fig2\07302023_refinedfigs\1_SwingDuration\FG_66Hz.svg', dpi = 300)
    plt.show()   

# Plot Heatmaps

In [ ]:
plot_heatmaps(BRK_66, 'BRK_66')

# Sort by swing/stance at lightON

In [ ]:
# All swing
BRK_66_L1swing, BRK_66_L2swing, BRK_66_L3swing, BRK_66_R1swing, BRK_66_R2swing, BRK_66_R3swing =  get_swing_at_lightON (BRK_66)
BRK_HL_66_L1swing, BRK_HL_66_L2swing, BRK_HL_66_L3swing, BRK_HL_66_R1swing, BRK_HL_66_R2swing, BRK_HL_66_R3swing =  get_swing_at_lightON (BRK_HL_66)
FG_66_L1swing, FG_66_L2swing, FG_66_L3swing, FG_66_R1swing, FG_66_R2swing, FG_66_R3swing =  get_swing_at_lightON (FG_66)
BB_66_L1swing, BB_66_L2swing, BB_66_L3swing, BB_66_R1swing, BB_66_R2swing, BB_66_R3swing =  get_swing_at_lightON (BB_66)
ES_66_L1swing, ES_66_L2swing, ES_66_L3swing, ES_66_R1swing, ES_66_R2swing, ES_66_R3swing =  get_swing_at_lightON (ES_66)

In [ ]:
# All stance
BRK_66_L1stance, BRK_66_L2stance, BRK_66_L3stance, BRK_66_R1stance, BRK_66_R2stance, BRK_66_R3stance =  get_stance_at_lightON (BRK_66)
BRK_HL_66_L1stance, BRK_HL_66_L2stance, BRK_HL_66_L3stance, BRK_HL_66_R1stance, BRK_HL_66_R2stance, BRK_HL_66_R3stance =  get_stance_at_lightON (BRK_HL_66)
FG_66_L1stance, FG_66_L2stance, FG_66_L3stance, FG_66_R1stance, FG_66_R2stance, FG_66_R3stance =  get_stance_at_lightON (FG_66)
BB_66_L1stance, BB_66_L2stance, BB_66_L3stance, BB_66_R1stance, BB_66_R2stance, BB_66_R3stance =  get_stance_at_lightON (BB_66)
ES_66_L1stance, ES_66_L2stance, ES_66_L3stance, ES_66_R1stance, ES_66_R2stance, ES_66_R3stance =  get_stance_at_lightON (ES_66)

plotting joint angles with step cycles

In [ ]:
plot_all_flexion_ang(BRK_66_L1swing, 'L1')

Swing Duration

In [ ]:
## note: some data points are removed after confirming that the swing duration was the result of bad tracking and not real stepping. 

In [ ]:
BRK_66_L1_swing_durdf, BRK_66_L1_swingdur_data = swing_dur_change(BRK_66_L1swing, 'L1')
BRK_66_L2_swing_durdf, BRK_66_L2_swingdur_data = swing_dur_change(BRK_66_L2swing, 'L2')
BRK_66_L3_swing_durdf, BRK_66_L3_swingdur_data = swing_dur_change(BRK_HL_66_L3swing, 'L3')
BRK_66_R1_swing_durdf, BRK_66_R1_swingdur_data = swing_dur_change(BRK_66_R1swing, 'R1')
BRK_66_R2_swing_durdf, BRK_66_R2_swingdur_data = swing_dur_change(BRK_66_R2swing, 'R2')
BRK_66_R3_swing_durdf, BRK_66_R3_swingdur_data = swing_dur_change(BRK_HL_66_R3swing, 'R3')


In [ ]:
S66_swing_dur_F = pd.concat([BRK_66_L1_swing_durdf.drop([]),BRK_66_R1_swing_durdf.drop([8])], axis = 0).reset_index(drop=True)
S66_swing_dur_M = pd.concat([BRK_66_L2_swing_durdf.drop([]),BRK_66_R2_swing_durdf.drop([])], axis = 0).reset_index(drop=True)
S66_swing_dur_H = pd.concat([BRK_66_L3_swing_durdf.drop([]),BRK_66_R3_swing_durdf.drop([])], axis =0).reset_index(drop=True)
plot_swing_dur_alllegs (S66_swing_dur_F, S66_swing_dur_M, S66_swing_dur_H)

In [ ]:
FG_66_L1_swing_durdf, FG_66_L1_swingdur_data = swing_dur_change(FG_66_L1swing, 'L1')
FG_66_L2_swing_durdf, FG_66_L2_swingdur_data = swing_dur_change(FG_66_L2swing, 'L2')
FG_66_L3_swing_durdf, FG_66_L3_swingdur_data = swing_dur_change(FG_66_L3swing, 'L3')
FG_66_R1_swing_durdf, FG_66_R1_swingdur_data = swing_dur_change(FG_66_R1swing, 'R1')
FG_66_R2_swing_durdf, FG_66_R2_swingdur_data = swing_dur_change(FG_66_R2swing, 'R2')
FG_66_R3_swing_durdf, FG_66_R3_swingdur_data = swing_dur_change(FG_66_R3swing, 'R3')

#### Plotting####

FG66_swing_dur_F = pd.concat([FG_66_L1_swing_durdf.drop([]),FG_66_R1_swing_durdf.drop([])], axis = 0)
FG66_swing_dur_M = pd.concat([FG_66_L2_swing_durdf.drop([]),FG_66_R2_swing_durdf.drop([])], axis = 0)
FG66_swing_dur_H = pd.concat([FG_66_L3_swing_durdf.drop([]),FG_66_R3_swing_durdf.drop([14])], axis =0)
plot_swing_dur_alllegs (FG66_swing_dur_F, FG66_swing_dur_M, FG66_swing_dur_H)

# Stopping parameters 

In [ ]:
def stopping_params_velthresh (df):
    Step_Count = {'L1' : [], 'L2' : [], 'L3' : [], 'R1' : [], 'R2': [] , 'R3' : []}
    Stopping_latency = pd.DataFrame()
    Stopping_bout_count = []
    init_bout_length = []
    Stopping_bout_duration = pd.DataFrame()
    
    stop_vel_x = 0.8

    avg_frames = 50


    for N in df['flynum'].unique(). tolist():
        for t in df.groupby('flynum').get_group(N)['tnum'].unique().tolist():
            data = df.groupby('flynum').get_group(N).groupby('tnum').get_group(t). reset_index(drop=True)
            x_vel = pd.DataFrame(abs(data['x_vel']))

            x_vel_smoothed = x_vel.iloc[400:1000,:].rolling(avg_frames, center= False).mean(). reset_index(drop=True)
            # x_vel_smoothed = x_vel.iloc[400:1000,:]. reset_index(drop=True)
            
            if len(x_vel_smoothed.loc[x_vel_smoothed['x_vel']<stop_vel_x])>0:

                init_stop_x_arr = np.where(np.array(abs(x_vel_smoothed)<stop_vel_x)==True)[0] 
                breakpts =np.split(init_stop_x_arr, np.where(np.diff(init_stop_x_arr)>20)[0]+1) 
                
                true_stop = []
                for i in range (len(breakpts)):
                    temp_bout = breakpts[i]
                    if len(temp_bout)>20:
                        true_stop.append(temp_bout)
                if (len(true_stop)>0):
                    num_bouts = len(true_stop)
                    first_stop = true_stop[0][0]
                    bout_dur = len(true_stop[0])
                    temp_latency = first_stop

                    pre_stop_data = data.iloc[400:400+first_stop, : ]

                    temp_stepcount_L1= np.count_nonzero((np.diff(pre_stop_data['L-F_stepcycle'].astype(int)))==1)
                    temp_stepcount_L2= np.count_nonzero((np.diff(pre_stop_data['L-M_stepcycle'].astype(int)))==1)
                    temp_stepcount_L3= np.count_nonzero((np.diff(pre_stop_data['L-H_stepcycle'].astype(int)))==1)
                    temp_stepcount_R1= np.count_nonzero((np.diff(pre_stop_data['R-F_stepcycle'].astype(int)))==1)
                    temp_stepcount_R2= np.count_nonzero((np.diff(pre_stop_data['R-M_stepcycle'].astype(int)))==1)
                    temp_stepcount_R3= np.count_nonzero((np.diff(pre_stop_data['R-H_stepcycle'].astype(int)))==1)
                
                    Step_Count['L1'].append(temp_stepcount_L1)
                    Step_Count['L2'].append(temp_stepcount_L2)
                    Step_Count['L3'].append(temp_stepcount_L3)
                    Step_Count['R1'].append(temp_stepcount_R1)
                    Step_Count['R2'].append(temp_stepcount_R2)
                    Step_Count['R3'].append(temp_stepcount_R3)

                    temp_latency_df = pd.DataFrame([N ,t, data['fnum'][400+first_stop], first_stop ]).T
                    Stopping_latency = pd.concat([Stopping_latency,temp_latency_df])
                    Stopping_bout_count.append(num_bouts)
                    init_bout_length.append(bout_dur)
                else:
                   num_bouts = 0 
                   Stopping_bout_count.append(num_bouts)
                   
                for i in range(len(true_stop)):
                    temp_bout_len = len(true_stop[i])
                    temp_bout_begin = true_stop[i][0]
                    temp_bout_end = true_stop[i][-1]
                    temp_bout_df = pd.DataFrame([N, t, temp_bout_begin,temp_bout_end, temp_bout_len]).T
                    Stopping_bout_duration = pd.concat([Stopping_bout_duration.reset_index(drop=True), temp_bout_df], axis = 0).reset_index(drop=True)


    Stopping_latency = pd.concat([Stopping_latency.reset_index(drop=True),pd.DataFrame(init_bout_length).reset_index(drop=True)], axis = 1 )
    Stopping_latency.columns = ['flynum', 'tnum', 'fnum', 'first_Stop', 'init_len']

    Stopping_bout_duration.columns = ['flynum', 'tnum', 'bout_begin', 'bount_end', 'bout_len']
    Stopping_bout_duration = Stopping_bout_duration.reset_index(drop=True)
    
    return Step_Count, Stopping_latency, Stopping_bout_count, Stopping_bout_duration

In [ ]:
Step_Count_BRK_66_1 , Stopping_latency_BRK_66_1, Stopping_bout_count_BRK_66_1, Stopping_bout_duration_BRK_66_1= stopping_params_velthresh(BRK_66)
Step_Count_FG_66_1, Stopping_latency_FG_66_1, Stopping_bout_count_FG_66_1, Stopping_bout_duration_FG_66_1 = stopping_params_velthresh(FG_66)
Step_Count_BB_66_1, Stopping_latency_BB_66_1, Stopping_bout_count_BB_66_1, Stopping_bout_duration_BB_66_1 = stopping_params_velthresh(BB_66)
Step_Count_ES_66_1, Stopping_latency_ES_66_1, Stopping_bout_count_ES_66_1, Stopping_bout_duration_ES_66_1 = stopping_params_velthresh(ES_66)

# Probability Density of FeTi flexion angle

In [ ]:
def plot_prob_density (dflist_swing, dflist_stance, angle, namelist = ['Stop1', 'FG'], colorlist = ['r', 'g']):
    plt.rcParams['figure.figsize'] = (10,5)
    fig, axs = plt.subplots(1,2, sharex = True, sharey = True)
    
    for gen_swing, color, name  in zip(dflist_swing, colorlist, namelist):
        
        count_df_swing = pd.DataFrame()

        for n in gen_swing['flynum'].unique().tolist():
            for t in gen_swing.groupby('flynum').get_group(n)['tnum'].unique().tolist():
                data = gen_swing.groupby('flynum').get_group(n).groupby('tnum').get_group(t).reset_index(drop=True)[400:1000]
                temp_ang = data[angle]
                count, bins = np.histogram(temp_ang, bins = np.arange(0,160,1), density = True)
                # plt.stairs(count, bins, color = color, fill = True, alpha = 0.5)
                count_df_swing = pd.concat([count_df_swing,pd.DataFrame(count)], axis = 1)

        y_swing = gaussian_filter1d(count_df_swing.mean(axis = 1), 5)
        sem = gaussian_filter1d((count_df_swing.std(axis = 1)/np.sqrt(len(count_df_swing.columns))), 5)

        axs[0].plot(y_swing, color = color, label = name)
        axs[0].fill_between(np.arange(0,159,1),y_swing-sem, y_swing+sem, alpha = 0.3, color = color)
        axs[0].set_title('Probability Density mid SWING : '+ angle)



    for gen_stance, color, name  in zip(dflist_stance, colorlist, namelist):

        count_df_stance = pd.DataFrame()

        for n in gen_stance['flynum'].unique().tolist():
            for t in gen_stance.groupby('flynum').get_group(n)['tnum'].unique().tolist():
                data = gen_stance.groupby('flynum').get_group(n).groupby('tnum').get_group(t).reset_index(drop=True)[400:1000]
                temp_ang = data[angle]
                count, bins = np.histogram(temp_ang, bins = np.arange(0,160,1), density = True)
                # plt.stairs(count, bins, color = color, fill = True, alpha = 0.5)
                count_df_stance = pd.concat([count_df_stance,pd.DataFrame(count)], axis = 1)

        y_stance = gaussian_filter1d(count_df_stance.mean(axis = 1), 5)
        sem = gaussian_filter1d((count_df_stance.std(axis = 1)/np.sqrt(len(count_df_stance.columns))), 5)

        axs[1].plot(y_stance, color = color, label = name)
        axs[1].fill_between(np.arange(0,159,1),y_stance-sem, y_stance+sem, alpha = 0.3, color = color)
        axs[1].set_title('Probability Density mid STANCE : '+ angle)
        plt.legend()
    plt.show()

In [ ]:
plot_prob_density([BRK_66_L1swing, FG_66_L1swing], [BRK_66_L1stance, FG_66_L1stance], 'L1C_flex')